## 一、综述

#### XGBoost 是 “Extreme Gradient Boosting” 的缩写，是boosting算法的其中一种。Boosting算法的思想是将许多弱分类器集成在一起形成一个强分类器。
#### 因为XGBoost是一种提升树模型，所以它是将许多树模型集成在一起，形成一个很强的分类器。而所用到的树模型则是CART回归树模型。
#### XGBoost是一个优化的分布式梯度增强库，旨在实现高效，灵活和便携。XGBoost提供并行树提升（也称为GBDT，GBM），可以快速准确地解决许多数据科学问题。
#### xgboost 的 model（模型） ： tree ensembles（树集成）。 树集成模型是一组 classification and regression trees （CART）。

## 二、原理详解

### 1.CART回归树

CART回归树是假设树为二叉树，通过不断将特征进行分裂。比如当前树结点是基于第j个特征值进行分裂的，设该特征值小于s的样本划分为左子树，大于s的样本划分为右子树。

CART回归树实质上就是在该特征维度对样本空间进行划分，而这种空间划分的优化是一种NP难问题，在决策树模型中是使用启发式方法解决。典型CART回归树产生的目标函数为：

$\sum_{x\epsilon R_m}(y_i-f(x_i))^2$

因此，当我们为了求解最优的切分特征j和最优的切分点s，就转化为求解这么一个目标函数：

$\min_{j,s}[min\sum_{x\epsilon(j,s)}(y_i-c_1)^2+min\sum_{x\epsilon(j,s)}(y_i-c_1)^2]$

所以我们只要遍历所有特征的的所有切分点，就能找到最优的切分特征和切分点。最终得到一棵回归树。

### 2.XGboost 总体思路

该算法思想就是不断地添加树，不断地进行特征分裂来生长一棵树，每次添加一个树，其实是学习一个新函数，去拟合上次预测的残差。

当我们训练完成得到k棵树，我们要预测一个样本的分数，其实就是根据这个样本的特征，在每棵树中会落到对应的一个叶子节点，每个叶子节点就对应一个分数，最后只需要将每棵树对应的分数加起来就是该样本的预测值。

### 3.目标函数解读

XGboost的目标函数是

$\sum_{i=1}^n(y_i-y_i^1)2$  ，$y_i^1$代表预测值

目标函数由两部分构成，第一部分用来衡量预测分数和真实分数的差距，另一部分则是正则化项。

新生成的树是要拟合上次预测的残差的，即当生成t棵树后，预测值会加上新树的预测值$f_t(x)$,接下来就是要去找到一个f_t能够最小化目标函数。

XGBoost的想法是利用其在f_t=0处的泰勒二阶展开近似它,而后得到的训练误差出去常数项就为最终得训练误差函数，具体如下

$L^t = \sum_{x=1}^n[g_if_i(x_i)+h_if_i^2(x_i)]$,$g_i,h_i$分别为$f_i$的一、二阶导数。

XGboost的正则化项如下：

$\gamma T+\lambda0.5\sum_{j=1}^Tw_j^2$


正则化项同样包含两部分，T表示叶子结点的个数，w表示叶子节点的分数。γ可以控制叶子结点的个数，λ可以控制叶子节点的分数不会过大，防止过拟合。

综上，XGboost的目标函数可改写为

$L = \sum{j=1}^t[(\sum{x\epsilon I_j}g_iw_j+0.5\sum{x\epsilon I_j}（h_i+\lambda）w_j^2]+\gamma T$

因此通过上式的改写，我们可以将目标函数改写成关于叶子结点分数w的一个一元二次函数，求解最优的w和目标函数值就变得很简单了，直接使用顶点公式即可。

因此最优解为

$w_j^* = \frac{G_i}{H_i+\lambda}$, $Obj = -\frac{1}{2}\sum_{j=1}^T\frac{G_i^2}{H_i+\lambda}+\gamma T$


### 4.确定树的结构

基于空间切分去构造一颗决策树是一个NP难问题，我们不可能去遍历所有树结构，因此，XGBoost使用了和CART回归树一样的想法，利用贪婪算法，遍历所有特征的所有特征划分点，不同的是使用上式目标函数值作为评价函数。

具体做法就是分裂后的目标函数值比单子叶子节点的目标函数的增益，同时为了限制树生长过深，还加了个阈值，只有当增益大于该阈值才进行分裂。

同时可以设置树的最大深度、当样本权重和小于设定阈值时停止生长去防止过拟合。

### 5.防止过拟合
XGBoost还提出了两种防止过拟合的方法：Shrinkage and Column Subsampling。

Shrinkage方法就是在每次迭代中对树的每个叶子结点的分数乘上一个缩减权重η，这可以使得每一棵树的影响力不会太大，留下更大的空间给后面生成的树去优化模型。

Column Subsampling类似于随机森林中的选取部分特征进行建树。

其可分为两种，一种是按层随机采样，在对同一层内每个结点分裂之前，先随机选择一部分特征，然后只需要遍历这部分的特征，来确定最优的分割点。

另一种是随机选择特征，则建树前随机选择一部分特征然后分裂就只遍历这些特征。一般情况下前者效果更好。

### 6.近似算法
对于连续型特征值，当样本数量非常大，该特征取值过多时，遍历所有取值会花费很多时间，且容易过拟合。

XGBoost思想是对特征进行分桶，即找到l个划分点，将位于相邻分位点之间的样本分在一个桶中。

在遍历该特征的时候，只需要遍历各个分位点，从而计算最优划分。

从算法伪代码中该流程还可以分为两种，全局的近似是在新生成一棵树之前就对各个特征计算分位点并划分样本，之后在每次分裂过程中都采用近似划分，而局部近似就是在具体的某一次分裂节点的过程中采用近似算法

### 7.缺省值处理

当样本的第i个特征值缺失时，无法利用该特征进行划分时，XGBoost的想法是将该样本分别划分到左结点和右结点，然后计算其增益，哪个大就划分到哪边。

## 三、调参策略

（1）	控制Bias-Variance权衡

（2）	控制过拟合

（3）	处理不平衡的数据集


## 四、XGboost优点

（1） 使用许多策略去防止过拟合，如：正则化项、Shrinkage and Column Subsampling等。  

（2）目标函数优化利用了损失函数关于待求函数的二阶导数   

（3）支持并行化，虽然树与树之间是串行关系，但是同层级节点可并行。具体的对于某个节点，节点内选择最佳分裂点，候选分裂点计算增益用多线程并行，训练速度快。  

（4）添加了对稀疏数据的处理。  

（5）交叉验证，early stop，当预测结果已经很好的时候可以提前停止建树，加快训练速度。  

（6）支持设置样本权重，该权重体现在一阶导数g和二阶导数h，通过调整权重可以去更加关注一些样本。  